# CSV 데이터 분석 노트북

통신사 가입자 정보 및 요금제 데이터 분석을 위한 Jupyter Notebook입니다.

## 데이터 파일 정보
- **ENTR_BY_INS.csv**: M-2 가입자 정보 (요금제)
- **ENTR_INT_INS.csv**: M-1 신규 가입자 정보 (요금제)
- **MVNO_PRD_PLC.csv**: 요금제 정보 (정책 관련 지원금 등)

In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'AppleGothic'  # macOS
plt.rcParams['axes.unicode_minus'] = False

# 출력 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("📚 라이브러리 로드 완료!")

## 1. 데이터 로드

In [ ]:
# CSV 파일들 로드
def load_csv_with_encoding(file_path, encodings=['utf-8', 'cp949', 'euc-kr']):
    """다양한 인코딩으로 CSV 파일 로드 시도"""
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"✅ {file_path} 로드 성공 (인코딩: {encoding})")
            return df, encoding
        except UnicodeDecodeError:
            continue
    raise ValueError(f"❌ {file_path} 로드 실패 - 지원되는 인코딩이 없습니다.")

# 데이터 로드
try:
    # M-2 가입자 정보
    df_m2, encoding_m2 = load_csv_with_encoding('csv/ENTR_BY_INS.csv', ['cp949', 'utf-8', 'euc-kr'])
    
    # M-1 신규 가입자 정보  
    df_m1, encoding_m1 = load_csv_with_encoding('csv/ENTR_INT_INS.csv', ['utf-8', 'cp949', 'euc-kr'])
    
    # 요금제 정보
    df_plan, encoding_plan = load_csv_with_encoding('csv/MVNO_PRD_PLC.csv', ['utf-8', 'cp949', 'euc-kr'])
    
    print("\n🎉 모든 데이터 로드 완료!")
    
except Exception as e:
    print(f"❌ 데이터 로드 중 오류: {e}")

## 2. 데이터 기본 정보 확인

In [3]:
# 데이터 기본 정보
datasets = {
    'M-2 가입자 정보 (ENTR_BY_INS)': df_m2,
    'M-1 신규 가입자 정보 (ENTR_INT_INS)': df_m1,
    '요금제 정보 (MVNO_PRD_PLC)': df_plan
}

for name, df in datasets.items():
    print(f"\n{'='*60}")
    print(f"📊 {name}")
    print(f"{'='*60}")
    print(f"📏 데이터 크기: {df.shape[0]:,}행 × {df.shape[1]}열")
    print(f"💾 메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"📋 컬럼 수: {len(df.columns)}개")
    
    # 결측값 정보
    missing_count = df.isnull().sum().sum()
    if missing_count > 0:
        print(f"⚠️  총 결측값: {missing_count:,}개")
    else:
        print(f"✅ 결측값 없음")

NameError: name 'df_m2' is not defined

## 3. M-2 가입자 정보 분석

In [ ]:
# M-2 가입자 정보 컬럼 확인
print("📋 M-2 가입자 정보 컬럼 목록:")
for i, col in enumerate(df_m2.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# M-2 가입자 정보 샘플 데이터
print("📄 M-2 가입자 정보 샘플 (상위 5개):")
display(df_m2.head())

In [ ]:
# M-2 가입자 정보 주요 통계
if '수납금액' in df_m2.columns:
    print("💰 수납금액 통계:")
    print(df_m2['수납금액'].describe())
    
    # 수납금액 분포 시각화
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    df_m2['수납금액'].hist(bins=50, alpha=0.7)
    plt.title('수납금액 분포')
    plt.xlabel('수납금액')
    plt.ylabel('빈도')
    
    plt.subplot(1, 2, 2)
    df_m2.boxplot(column='수납금액')
    plt.title('수납금액 박스플롯')
    
    plt.tight_layout()
    plt.show()

## 4. M-1 신규 가입자 정보 분석

In [ ]:
# M-1 신규 가입자 정보 컬럼 확인
print("📋 M-1 신규 가입자 정보 컬럼 목록:")
for i, col in enumerate(df_m1.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# M-1 신규 가입자 정보 샘플 데이터
print("📄 M-1 신규 가입자 정보 샘플 (상위 5개):")
display(df_m1.head())

In [ ]:
# 개통유형별 분석
if '개통유형' in df_m1.columns:
    print("📊 개통유형별 분포:")
    activation_counts = df_m1['개통유형'].value_counts()
    print(activation_counts)
    
    # 개통유형 파이차트
    plt.figure(figsize=(8, 6))
    plt.pie(activation_counts.values, labels=activation_counts.index, autopct='%1.1f%%')
    plt.title('개통유형별 분포')
    plt.show()

## 5. 요금제 정보 분석

In [ ]:
# 요금제 정보 전체 보기
print("📋 요금제 정보:")
display(df_plan)

In [ ]:
# 요금제별 기본료 분석
if '기본료' in df_plan.columns:
    print("💰 요금제별 기본료:")
    print(df_plan[['요금제명', '기본료']].to_string(index=False))
    
    # 기본료 시각화
    if len(df_plan) > 1:
        plt.figure(figsize=(10, 6))
        plt.bar(range(len(df_plan)), df_plan['기본료'])
        plt.xticks(range(len(df_plan)), df_plan['요금제명'], rotation=45, ha='right')
        plt.title('요금제별 기본료')
        plt.ylabel('기본료 (원)')
        plt.tight_layout()
        plt.show()

## 6. 데이터 간 관계 분석

In [ ]:
# M-2 데이터에서 요금제별 분석
if 'MVNO상품명' in df_m2.columns:
    print("📊 M-2 데이터 - 요금제별 가입자 수:")
    plan_counts = df_m2['MVNO상품명'].value_counts()
    print(plan_counts)
    
    # 요금제별 가입자 수 시각화
    plt.figure(figsize=(12, 6))
    plan_counts.plot(kind='bar')
    plt.title('요금제별 가입자 수 (M-2 데이터)')
    plt.xlabel('요금제명')
    plt.ylabel('가입자 수')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# M-1 데이터에서 요금제별 분석
if '현재요금제명' in df_m1.columns:
    print("📊 M-1 데이터 - 요금제별 신규 가입자 수:")
    new_plan_counts = df_m1['현재요금제명'].value_counts()
    print(new_plan_counts)
    
    # 요금제별 신규 가입자 수 시각화
    plt.figure(figsize=(12, 6))
    new_plan_counts.plot(kind='bar')
    plt.title('요금제별 신규 가입자 수 (M-1 데이터)')
    plt.xlabel('요금제명')
    plt.ylabel('신규 가입자 수')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 7. 인터랙티브 시각화 (Plotly)

In [ ]:
# M-2 데이터 인터랙티브 시각화
if 'MVNO상품명' in df_m2.columns and '수납금액' in df_m2.columns:
    # 요금제별 수납금액 분포
    fig = px.box(df_m2, x='MVNO상품명', y='수납금액', 
                 title='요금제별 수납금액 분포',
                 labels={'MVNO상품명': '요금제명', '수납금액': '수납금액 (원)'})
    fig.update_xaxes(tickangle=45)
    fig.show()
    
    # 수납금액 히스토그램
    fig = px.histogram(df_m2, x='수납금액', nbins=50,
                       title='수납금액 분포 히스토그램',
                       labels={'수납금액': '수납금액 (원)', 'count': '빈도'})
    fig.show()

## 8. 데이터 요약 및 인사이트

In [ ]:
# 전체 데이터 요약
print("📊 전체 데이터 요약")
print("=" * 50)

total_m2_users = len(df_m2)
total_m1_users = len(df_m1)
total_plans = len(df_plan)

print(f"📈 M-2 가입자 수: {total_m2_users:,}명")
print(f"📈 M-1 신규 가입자 수: {total_m1_users:,}명")
print(f"📋 총 요금제 수: {total_plans}개")

if '수납금액' in df_m2.columns:
    total_revenue = df_m2['수납금액'].sum()
    avg_revenue = df_m2['수납금액'].mean()
    print(f"💰 총 수납금액: {total_revenue:,.0f}원")
    print(f"💰 평균 수납금액: {avg_revenue:,.0f}원")

print("\n🎯 주요 인사이트:")
print("- 데이터 분석을 통해 요금제별 가입자 현황 파악 가능")
print("- 신규 가입자와 기존 가입자 패턴 비교 분석 가능")
print("- 수익성 분석 및 요금제 최적화 방향 도출 가능")